In [1]:
import cobra
import libsbml
import escher
from cobra import Model, Reaction, Metabolite
import csv
import pandas as pd
import copy
import cplex

In [2]:
model = cobra.io.read_sbml_model('../model/ios2164_2cell.xml')

In [3]:
model.solver = 'cplex'

In [4]:
read = pd.read_csv('../misc/photo_medium.csv')

photo_medium = dict(read.values)

for item in list(photo_medium.keys()):
    if "_V" in item:
        photo_medium.pop(item)

photo_medium

{'EX_co2_LPAREN_e_RPAREN__M': 1000,
 'EX_h2o_LPAREN_e_RPAREN__M': 1000,
 'EX_h_LPAREN_e_RPAREN__M': 1000,
 'EX_no3_LPAREN_e_RPAREN__M': 1000,
 'EX_o2_LPAREN_e_RPAREN__M': 1000,
 'EX_pi_LPAREN_e_RPAREN__M': 1000,
 'EX_sucr_LPAREN_e_RPAREN__M': 0,
 'EX_fru_DASH_B_LPAREN_e_RPAREN__M': 0,
 'EX_glc_DASH_A_LPAREN_e_RPAREN__M': 0,
 'EX_so4_LPAREN_e_RPAREN__M': 1000,
 'EX_hco3_LPAREN_e_RPAREN__M': 0,
 'EX_so3_LPAREN_e_RPAREN__M': 1000,
 'EX_h2s_LPAREN_e_RPAREN__M': 0,
 'EX_nh4_LPAREN_e_RPAREN__M': 0,
 'EX_asn_DASH_L_LPAREN_e_RPAREN__M': 0,
 'EX_gln_DASH_L_LPAREN_e_RPAREN__M': 0,
 'EX_etoh_LPAREN_e_RPAREN__M': 0,
 'EX_ac_LPAREN_e_RPAREN__M': 0,
 'EX_lac_DASH_L_LPAREN_e_RPAREN__M': 0,
 'EX_tsul_LPAREN_e_RPAREN__M': 0,
 'EX_fe2_LPAREN_e_RPAREN__M': 1000,
 'EX_fe3_LPAREN_e_RPAREN__M': 0,
 'EX_mg2_LPAREN_e_RPAREN__M': 1000,
 'EX_ala_DASH_L_LPAREN_e_RPAREN__M': 0,
 'EX_arg_DASH_L_LPAREN_e_RPAREN__M': 0,
 'EX_asp_DASH_L_LPAREN_e_RPAREN__M': 0,
 'EX_cys_DASH_L_LPAREN_e_RPAREN__M': 0,
 'EX_glu_DASH_L_L

In [5]:

model.medium = photo_medium
model.medium

#Model only has gas exchange, nitrate, phosphate, sulfate, and other inorganic substances as input plus light)

{'EX_co2_LPAREN_e_RPAREN__M': 1000,
 'EX_h2o_LPAREN_e_RPAREN__M': 1000,
 'EX_h_LPAREN_e_RPAREN__M': 1000,
 'EX_no3_LPAREN_e_RPAREN__M': 1000,
 'EX_o2_LPAREN_e_RPAREN__M': 1000,
 'EX_pi_LPAREN_e_RPAREN__M': 1000,
 'EX_so4_LPAREN_e_RPAREN__M': 1000,
 'EX_so3_LPAREN_e_RPAREN__M': 1000,
 'EX_fe2_LPAREN_e_RPAREN__M': 1000,
 'EX_mg2_LPAREN_e_RPAREN__M': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__M': 1000,
 'EX_h2o_LPAREN_e_RPAREN__BS': 1000,
 'EX_h_LPAREN_e_RPAREN__BS': 1000,
 'EX_no3_LPAREN_e_RPAREN__BS': 1000,
 'EX_pi_LPAREN_e_RPAREN__BS': 1000,
 'EX_so4_LPAREN_e_RPAREN__BS': 1000,
 'EX_fe2_LPAREN_e_RPAREN__BS': 1000,
 'EX_mg2_LPAREN_e_RPAREN__BS': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__BS': 1000}

In [6]:
#What if I changed light flux from 1000 to 200?
photo_medium['EX_photonVis_LPAREN_e_RPAREN__M'] = 200
photo_medium['EX_photonVis_LPAREN_e_RPAREN__BS'] = 200
model.medium = photo_medium
model.summary()

#Kaya pala, kelangan ko i-constrain specifically yung CO2 flux sa BS to 0 rather than 
#defining it explicitly sa medium lang

Metabolite,Reaction,Flux,C-Number,C-Flux
h_e,EX_h_LPAREN_e_RPAREN__BS,1000,0,0.00%
h_e,EX_h_LPAREN_e_RPAREN__M,1000,0,0.00%
no3_e,EX_no3_LPAREN_e_RPAREN__BS,9.596,0,0.00%
o2_e,EX_o2_LPAREN_e_RPAREN__M,1000,0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__BS,200,0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__M,200,0,0.00%
pi_e,EX_pi_LPAREN_e_RPAREN__M,3.09,0,0.00%
hso3_e,EX_so3_LPAREN_e_RPAREN__M,581.2,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
coa_c1,DM_COA_LPAREN_c_RPAREN__BS,-7.938E-05,21,52.52%


In [7]:
model.reactions.CO2tex_BS.bounds = (-1000, 0)
model.reactions.CO2tex_M.bounds = (-1000, 20)
model.reactions.O2tex_BS.bounds = (-1000, 0)
model.reactions.O2tex_M.bounds = (-1000, 1000)
model.metabolites.o2_c1.summary()

#Positive values will mean net flux in the forward direction; negative values means in reverse.
#So ibig sabihin there's net release of CO2 papunta sa extracellular environment. Ok
#I think this is correct na, I need to set up a gas exchange mechanism where the partial pressure
#Of O2 needs to be linked with the partial pressure of CO2

Percent,Flux,Reaction,Definition
18.85%,90.16,O2tv_BS,o2_c1 <=> o2_v1
81.15%,388.2,o2_pd,o2_c0 <=> o2_c1
Percent,Flux,Reaction,Definition
0.40%,-1.898,CC3H1c_BS,coum_c1 + nadph_c1 + o2_c1 --> caff_c1 + h2o_c1 + nadp_c1
0.01%,-0.02533,FAS161COA_Lc_BS,nadph_c1 + o2_c1 + pmtcoa_c1 --> 2.0 h2o_c1 + hdcoa_c1 + nadp_c1
0.09%,-0.4349,FAS181COA_Lc_BS,nadph_c1 + o2_c1 + stcoa_c1 --> 2.0 h2o_c1 + nadp_c1 + odecoa_c1
0.05%,-0.2255,FAS182COA_Lc_BS,nadph_c1 + o2_c1 + odecoa_c1 --> 2.0 h2o_c1 + nadp_c1 + ocdycacoa_c1
0.00%,-0.01467,FAS183COA_Lc_BS,nadph_c1 + o2_c1 + ocdycacoa_c1 --> 2.0 h2o_c1 + lnlncoa_c1 + nadp_c1
0.18%,-0.8758,FERH2c_BS,conald_c1 + h_c1 + nadph_c1 + o2_c1 --> h2o_c1 + hconald_c1 + nadp_c1
74.84%,-358,O2tmr_BS,o2_c1 <=> o2_m1


In [10]:
model.reactions.CO2tex_BS

Reaction identifier,CO2tex_BS
Name,CO2 transport (extracellular to cytosol)
Memory address,0x07fcdb0e2f2e0
Stoichiometry,co2_e <-- co2_c1 CO2 <-- CO2
GPR,
Lower bound,-1000
Upper bound,0


In [58]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
h_e,EX_h_LPAREN_e_RPAREN__BS,1000,0,0.00%
h_e,EX_h_LPAREN_e_RPAREN__M,1000,0,0.00%
no3_e,EX_no3_LPAREN_e_RPAREN__M,9.596,0,0.00%
o2_e,EX_o2_LPAREN_e_RPAREN__M,1000,0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__BS,200,0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__M,200,0,0.00%
pi_e,EX_pi_LPAREN_e_RPAREN__M,3.09,0,0.00%
hso3_e,EX_so3_LPAREN_e_RPAREN__M,581.2,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
coa_c1,DM_COA_LPAREN_c_RPAREN__BS,-5.994E-05,21,39.66%


In [32]:
model.metabolites.o2_c1.summary()

Percent,Flux,Reaction,Definition
18.85%,90.16,O2tv_BS,o2_c1 <=> o2_v1
81.15%,388.2,o2_pd,o2_c0 <=> o2_c1
Percent,Flux,Reaction,Definition
0.40%,-1.898,CC3H1c_BS,coum_c1 + nadph_c1 + o2_c1 --> caff_c1 + h2o_c1 + nadp_c1
0.01%,-0.02533,FAS161COA_Lc_BS,nadph_c1 + o2_c1 + pmtcoa_c1 --> 2.0 h2o_c1 + hdcoa_c1 + nadp_c1
0.09%,-0.4349,FAS181COA_Lc_BS,nadph_c1 + o2_c1 + stcoa_c1 --> 2.0 h2o_c1 + nadp_c1 + odecoa_c1
0.05%,-0.2255,FAS182COA_Lc_BS,nadph_c1 + o2_c1 + odecoa_c1 --> 2.0 h2o_c1 + nadp_c1 + ocdycacoa_c1
0.00%,-0.01467,FAS183COA_Lc_BS,nadph_c1 + o2_c1 + ocdycacoa_c1 --> 2.0 h2o_c1 + lnlncoa_c1 + nadp_c1
0.18%,-0.8758,FERH2c_BS,conald_c1 + h_c1 + nadph_c1 + o2_c1 --> h2o_c1 + hconald_c1 + nadp_c1
74.84%,-358,O2tmr_BS,o2_c1 <=> o2_m1


In [38]:
model.metabolites.co2_c1.summary()

Percent,Flux,Reaction,Definition
1.47%,5.909,CO2tm_BS,co2_c1 <=> co2_m1
0.01%,0.02019,CO2tr_BS,co2_c1 <=> co2_r1
89.31%,359.8,FDHNc_BS,for_c1 + 2.0 h_m1 + q8_m1 --> co2_c1 + h_c1 + q8h2_m1
0.05%,0.1991,PPDHc_BS,h_c1 + pphn_c1 --> co2_c1 + h2o_c1 + phpyr_c1
1.03%,4.142,UGDCc_BS,h_c1 + udpglcur_c1 --> co2_c1 + udpxyl_c1
8.14%,32.8,co2_pd,co2_c0 <=> co2_c1
Percent,Flux,Reaction,Definition
96.53%,-388.9,CO2ts_BS,co2_c1 <=> co2_s1
1.18%,-4.749,HCO3Ec_BS,h_c1 + hco3_c1 <=> co2_c1 + h2o_c1
2.29%,-9.224,PPCc_BS,co2_c1 + h2o_c1 + pep_c1 <=> h_c1 + oaa_c1 + pi_c1


In [46]:
model.metabolites.sucr_c0.summary()

Percent,Flux,Reaction,Definition
100.00%,2.022,sucr_pd,sucr_c0 <=> sucr_c1
Percent,Flux,Reaction,Definition
53.13%,-1.074,SUSc_M,fru_DASH_B_c0 + udpg_c0 <=> h_c0 + sucr_c0 + udp_c0
46.87%,-0.9477,Straw_Biomass_M,0.0321 ala_DASH_L_c0 + 0.0123 arg_DASH_L_c0 + 0.0417 asp_DASH_L_c0 + 33.259 atp_c0 + 2.0789 cellulose_c0 + 0.1721 conoh_c0 + 0.2065 coumoh_c0 + 0.0071 ctp_c0 + 0.0033 cys_DASH_L_c0 + 0.0009 datp_c0 + 0.0007 dctp_c0 + 0.0007 dgtp_c0 + 0.0009 dttp_c0 + 0.00285 fa_os_r0 + 0.0643 fru_DASH_B_c0 + 0.0546 glc_DASH_A_c0 + 0.0385 glu_DASH_L_c0 + 0.0282 gly_c0 + 0.0071 gtp_c0 + 0.057 his_DASH_L_c0 + 0.014 ile_DASH_L_c0 + 0.0234 leu_DASH_L_c0 + 0.0153 lys_DASH_L_c0 + 0.0076 met_DASH_L_c0 + 0.000191 pa_os_r0 + 0.0034 pc_os_r0 + 0.00178 pe_os_r0 + 0.0133 phe_DASH_L_c0 + 0.0225 pro_DASH_L_c0 + 0.00041 ptd1ino_os_r0 + 0.0219 ser_DASH_L_c0 + 0.1475 sinoh_c0 + 0.3249 starch_s0 + 0.1049 sucr_c0 + 0.0192 thr_DASH_L_c0 + 0.0298 triglyc_os_r0 + 0.0016 trp_DASH_L_c0 + 0.0077 tyr_DASH_L_c0 + 0.1256 udparab_c0 + 0.4018 udpg_c0 + 0.0229 udpgal_c0 + 0.5033 udpglcur_c0 + 0.572 udpxyl_c0 + 0.009 utp_c0 + 0.0225 val_DASH_L_c0 --> 33.259 adp_c0 + 1.626 h_c0 + 33.259 pi_c0 + 1.626 udp_c0


In [33]:
model.metabolites.get_by_id('atp_c1').summary()
#No import of CO2 from the environment, so far so good

Percent,Flux,Reaction,Definition
0.07%,0.7586,ATP2tr_BS,amp_r1 + atp_c1 + h_r1 <=> amp_c1 + atp_r1 + h_c1
70.24%,815.5,ATPtm_BS,adp_c1 + atp_m1 + h_c1 --> adp_m1 + atp_c1 + h_m1
0.00%,0.005344,DUMPKc_BS,atp_c1 + dump_c1 <=> adp_c1 + dudp_c1
29.69%,344.7,FTHFLc_BS,atp_c1 + for_c1 + thf_c1 <-- 10fthf_c1 + adp_c1 + pi_c1
Percent,Flux,Reaction,Definition
0.32%,-3.728,ADK1c_BS,amp_c1 + atp_c1 --> 2.0 adp_c1
0.06%,-0.7545,ATCDc_BS,atp_c1 + cdp_c1 --> adp_c1 + ctp_c1
0.00%,-0.04216,ATCYc_BS,atp_c1 + cytd_c1 --> adp_c1 + cmp_c1 + h_c1
0.00%,-0.005344,ATDADc_BS,atp_c1 + dadp_c1 --> adp_c1 + datp_c1
0.00%,-0.004156,ATDCDc_BS,atp_c1 + dcdp_c1 --> adp_c1 + dctp_c1


In [34]:
model.reactions.ATP2tr_M

Reaction identifier,ATP2tr_M
Name,"AMP/ATP transporter, ER"
Memory address,0x07ff6a819cdf0
Stoichiometry,amp_r0 + atp_c0 + h_r0 <=> amp_c0 + atp_r0 + h_c0 AMP + ATP + H+ <=> AMP + ATP + H+
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [35]:
from cobra.test import create_test_model
from cobra.sampling import sample

%time s = cobra.flux_analysis.flux_variability_analysis(model, processes = 4)
s

/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"

CPU times: user 220 ms, sys: 56.7 ms, total: 277 ms
Wall time: 8.96 s


,minimum,maximum
Coleoptile_Biomass_M,0.000000e+00,0.000000
Straw_Biomass_M,0.000000e+00,14.971478
ACCOAC_OSAc_M,0.000000e+00,0.000000
ACCOAC_OSAr_M,0.000000e+00,1.511550
ACPC_OSs_M,0.000000e+00,0.000000
...,...,...
g3p_pd,1.071058e-06,0.000003
3pg_pd,1.071058e-06,0.000003
glyclt_pd,1.071058e-06,0.000003
acon_DASH_C_pd,1.596784e-07,0.000003


In [36]:
%time model.summary(fva=0.90)

/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


CPU times: user 443 ms, sys: 108 ms, total: 551 ms
Wall time: 1.66 s


Metabolite,Reaction,Flux,Range,C-Number,C-Flux
h_e,EX_h_LPAREN_e_RPAREN__BS,1000,[-494.5; 1000],0,0.00%
h_e,EX_h_LPAREN_e_RPAREN__M,1000,[-494.5; 1000],0,0.00%
no3_e,EX_no3_LPAREN_e_RPAREN__M,9.596,[0; 131.3],0,0.00%
o2_e,EX_o2_LPAREN_e_RPAREN__M,1000,[783.7; 1000],0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__BS,200,[0; 200],0,0.00%
photonVis_e,EX_photonVis_LPAREN_e_RPAREN__M,200,[0; 200],0,0.00%
pi_e,EX_pi_LPAREN_e_RPAREN__M,3.09,[-997.2; 1000],0,0.00%
hso3_e,EX_so3_LPAREN_e_RPAREN__M,581.2,[0; 1000],0,0.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
3_DASH_nond_s1,DM_3_DASH_NOND_LPAREN_s_RPAREN__BS,0,[-4.223; 0],9,0.00%


In [37]:
#This shows the plasmodesmatal transport reactions present in the rice cell

pfba_soln = cobra.flux_analysis.pfba(model)

for item in model.reactions:
    if "_pd" in item.id:
        print(item.id, "|", pfba_soln[item.id])

ala_DASH_L_pd | 0.0
arg_DASH_L_pd | 0.07302970165859787
asp_DASH_L_pd | 0.0
cys_DASH_L_pd | 0.0647173778925786
glu_DASH_L_pd | -0.3250110155621557
gly_pd | 2.8646514954697606
his_DASH_L_pd | 0.338430324759356
ile_DASH_L_pd | -0.12647745330150065
leu_DASH_L_pd | -0.21139802908965108
lys_DASH_L_pd | -0.1382217882509257
met_DASH_L_pd | 0.0
phe_DASH_L_pd | -0.12015358063642562
pro_DASH_L_pd | 0.0
ser_DASH_L_pd | 0.41622106275049076
thr_DASH_L_pd | 0.0
trp_DASH_L_pd | -0.014454566091600075
tyr_DASH_L_pd | -4.822404612310075
val_DASH_L_pd | -0.20326733566312605
glc_DASH_A_pd | 0.0
sucr_pd | -2.0218324320625602
fru_DASH_B_pd | 0.0
udparab_pd | 0.0
udpxyl_pd | 0.0
udpgal_pd | 0.0
udpg_pd | 0.0
udpglcur_pd | 0.10271657225152264
co2_pd | 32.804397089175716
h2o_pd | 0.0
o2_pd | 388.20740833905006
pep_pd | 0.0
pyr_pd | 0.0
mal_DASH_L_pd | -1.2880611740814638
accoa_pd | 0.0
glu5p_pd | 0.0
akg_pd | 0.0
icit_pd | 0.0
gal_pd | 0.0
g1p_pd | 0.0
tre_pd | 0.0
glc_DASH_B_pd | 0.0
tre6p_pd | 0.0
g6p_DASH_A

SyntaxError: invalid syntax (<ipython-input-16-e5c9403477cf>, line 1)